<a href="https://colab.research.google.com/github/dshinde/DSTestGitHub/blob/master/IMageClassificationUSPresident.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 1. Setup and Load Data

Define hyperparameters
num_classes = 4 batch_size = 16 epochs = 10 learning_rate = 0.001 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Define data transformations
transform = transforms.Compose([ transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

Load dataset (Assuming dataset folder structure: data/train and data/val)
train_dataset = datasets.ImageFolder(root="data/train", transform=transform) val_dataset = datasets.ImageFolder(root="data/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## 1.1 Install Dependencies and Setup

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data directories
data_dir = "/content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data"  # Replace with your dataset directory
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

# Define transformations
transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

## 1.2 Remove Dodgy Images from train and val folders

In [7]:
#Opensource library for computer vision
import cv2
# Checks file extension if it is valid image
import imghdr
image_exts = ['jpeg','jpg','bmp','png']

## You will have to run it twice for train and val
for image_class in os.listdir(val_dir):
    for image in os.listdir(os.path.join(val_dir, image_class)):
        image_path = os.path.join(val_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

image_exts = ['jpeg','jpg','bmp','png']


Image not in ext list /content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data/val/fdr/Franklin-D-Roosevelt-1937.jpg
Image not in ext list /content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data/val/obama/overview-Barack-Obama.jpg
Image not in ext list /content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data/val/obama/Michelle-Obama-portrait-first-lady-Blue-Room-2009.jpg
Image not in ext list /content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data/val/obama/MoS_PresidentBarackObama_colorcutout-600x600.webp
Image not in ext list /content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data/val/lincoln/small-archival-canvas-10195-24412-fm400.jpg
Image not in ext list /content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data/val/lincoln/President-Abraham-Lincoln-painting-George-Healy-1887-National-Portrait-Gallery.jpg
Image not

## 1.3 Load Data

In [8]:
# Datasets and DataLoaders
train_dataset = datasets.ImageFolder(train_dir, transform=transform["train"])
val_dataset = datasets.ImageFolder(val_dir, transform=transform["val"])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## 1.4 Show Train and Val impages on Plot

# 2. Train and Eval Model

In [9]:
# Load pre-trained model
model = models.resnet18(pretrained=True)

# Modify the last layer for 3 classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = correct / total

        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_loss = val_loss / len(val_dataset)
        val_acc = correct / total

        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}\n")

    return model

# Train the model
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 137MB/s]


Epoch 1/10
----------
Train Loss: 0.6123 Acc: 0.7347
Val Loss: 14.3958 Acc: 0.2214

Epoch 2/10
----------
Train Loss: 0.9385 Acc: 0.7704
Val Loss: 20.0072 Acc: 0.1643

Epoch 3/10
----------
Train Loss: 0.2049 Acc: 0.9133
Val Loss: 0.8431 Acc: 0.7929

Epoch 4/10
----------
Train Loss: 0.3532 Acc: 0.8571
Val Loss: 2.2282 Acc: 0.2571

Epoch 5/10
----------
Train Loss: 0.4417 Acc: 0.8214
Val Loss: 0.7867 Acc: 0.7429

Epoch 6/10
----------
Train Loss: 0.2493 Acc: 0.9184
Val Loss: 1.1140 Acc: 0.6929

Epoch 7/10
----------
Train Loss: 0.1570 Acc: 0.9694
Val Loss: 1.7249 Acc: 0.5357

Epoch 8/10
----------
Train Loss: 0.3990 Acc: 0.9133
Val Loss: 0.7894 Acc: 0.7643

Epoch 9/10
----------
Train Loss: 0.3248 Acc: 0.8776
Val Loss: 0.4591 Acc: 0.8714

Epoch 10/10
----------
Train Loss: 0.2006 Acc: 0.9235
Val Loss: 0.4500 Acc: 0.8500



# 3. Save model

In [12]:
# Save the model
model_path = "president_classifier.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to president_classifier.pth


# 4. Test Model

In [15]:
# Test the model with a sample image
def test_model(model, image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = transform_test(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    classes = train_dataset.classes
    print(f"Predicted: {classes[preds.item()]}")

# Test the model on a sample image
test_image_path = "/content/drive/MyDrive/Colab Notebooks/CNN Samples/PresidentImageClassification/data/FDRTest.jpg"  # Replace with your test image path
model.load_state_dict(torch.load(model_path))
test_model(model, test_image_path)


<ipython-input-15-3e9e327bfca2>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Predicted: fdr
